In [ ]:
changing_parameter = [-3.,-3.,-0.8,-0.3,0.5,0.6]
from Chempy.cem_function import posterior_function_for_integration

In [ ]:
%timeit -r 10 posterior_function_for_integration(changing_parameter)

## DEFINITELY CAN IMPROVE THIS

In [ ]:
def posterior_function_for_integration(changing_parameter):
	'''
	This is the actual posterior function. But the functionality is explained in posterior_function. 
	This is a cut down version for integration - ONLY using beta function, and solar data
	'''
	from .parameter import ModelParameters
	from .cem_function import posterior_function_returning_predictions
	from .data_to_test import likelihood_function_for_integration
		
	a = ModelParameters()
	stellar_identifier = a.stellar_identifier
	
	# the values in a are updated according to changing_parameters and the prior list is appended
	a = extract_parameters_and_priors(changing_parameter, a)
	
	# the log prior is calculated
	prior = sum(np.log(a.prior))

	#_,elements = posterior_function_returning_predictions((changing_parameter,a))
	
	# call Chempy and return the abundances at the end of the simulation = time of star's birth and the corresponding element names as a list
	abundance_list,elements_to_trace = cem_real2(a)
	
	# The last two entries of the abundance list are the Corona metallicity and the SN-ratio
	abundance_list = abundance_list[:-2]
	elements_to_trace = elements_to_trace[:-2]

    wildcard = np.load('Chempy/input/stars/Proto-sun.npy')	

	 

	# a likelihood is calculated where the model error is optimized analytically if you do not want model error uncomment one line in the likelihood function
	from scipy.stats import beta
	likelihood_list = []
	model_errors = np.linspace(a.flat_model_error_prior[0],a.flat_model_error_prior[1],a.flat_model_error_prior[2])
	error_weight = beta.pdf(model_errors, a = a.beta_error_distribution[1], b = a.beta_error_distribution[2])
	error_weight/= sum(error_weight)

	
	for i, item in enumerate(model_errors):
		error_temp = np.ones(len(elements)) * item 
		likelihood_temp, _, _, _, _, _ = likelihood_function_for_integration(wildcard, abundance_list, elements_to_trace, fixed_model_error = error_temp, elements = elements)
		likelihood_list.append(likelihood_temp)
	likelihood = logsumexp(likelihood_list, b = error_weight)

	return(prior+likelihood)

In [ ]:
from Chempy.cem_function import posterior_function_returning_predictions

In [1]:
%load_ext line_profiler

In [2]:
from Chempy.cem_function import posterior_function_for_integration
from Chempy.score_function import preload_params
b = preload_params()
changing_parameter = [-3.,-3.,-0.8,-0.3,0.5,0.6]

In [3]:
from Chempy.cem_function import cem_real2_int, extract_parameters_and_priors
from Chempy.parameter import ModelParameters
a = ModelParameters
a = extract_parameters_and_priors(changing_parameter,a)
%timeit cem_real2_int(a,b)

100 loops, best of 3: 3.55 ms per loop


In [4]:
abundance_list = cem_real2_int(a,b)

# The last two entries of the abundance list are the Corona metallicity and the SN-ratio
list_of_abundances = abundance_list[:-2]
abundance_list = []
#star_abundance_list = []
#star_error_list = []
# Change to arrays to speed up?
for i,item in enumerate(a.elements_to_trace):
    if item in list(b.wildcard.dtype.names):
        #elements.append(item)
        abundance_list.append(float(list_of_abundances[i]))


In [5]:
from Chempy.data_to_test import likelihood_evaluation_int
import numpy as np
%lprun -f likelihood_evaluation_int likelihood_evaluation_int(b.errors_list[0],abundance_list,b.star_abundance_list)
#%timeit likelihood_evaluation(error_temp,b.star_error_list,abundance_list,b.star_abundance_list)


In [6]:
%timeit -r 10 -n 10 posterior_function_for_integration(changing_parameter,b)

10 loops, best of 10: 8.36 ms per loop


In [7]:
%lprun -f posterior_function_for_integration posterior_function_for_integration(changing_parameter,b)

In [ ]:
%lprun -f cem_real2_int cem_real2_int(a,b)